## Classifying Audio Vs Visual labels 

In [ ]:
%run ../preprocessing/Artifact_Removal/preprocecssing_helpers.ipynb
%run ../preprocessing/Artifact_Removal/Extract_Describer_Events.ipynb
%run ../preprocessing/StimCodes.ipynb
%run ../Classification/ConcatEpochTrails.ipynb
%run ../PCA/Emmanuil-PCA.ipynb
import mne 
import sys
from mne import io, EvokedArray
from mne.decoding import Vectorizer, get_coef
from sklearn.preprocessing import MinMaxScaler, Normalizer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.svm import SVC  # noqa
from sklearn.model_selection import ShuffleSplit  # noqa

from mne.decoding import UnsupervisedSpatialFilter, SlidingEstimator,cross_val_multiscore
from sklearn.decomposition import PCA

from sklearn.metrics import confusion_matrix
from mne.viz import tight_layout
from mne.decoding import LinearModel
import matplotlib.pyplot as plt
import numpy as np
matplotlib.rcParams['figure.figsize'] = (15, 15)
matplotlib.rcParams.update({'font.size': 15})
%matplotlib inline
import time

byteToGB = 1000000000

In [ ]:
# path = 'E:/correct_time_epochs/'
path = '/sharedData3/proj_mdg/scratch-eproj/NEW_EPOCH_DATA/'
epoch_files = os.listdir(path)
epoch_files[:5]
file = os.path.join(path, epoch_files[0]) #19

In [ ]:
epoch_files;

In [ ]:
# read epoch object 
epoch = mne.read_epochs(file, preload= True)

In [ ]:
epochs_list = []
total_bytes = 0
master_epochs_list = []
for file in epoch_files:
    if total_bytes/byteToGB > 4:
        master_epochs_list.append(epochs_list)
        epochs_list = []
        total_bytes = 0
    else:
        epoch_path = os.path.join(path, file)
        epoch = mne.read_epochs(epoch_path, preload = True)
        new_epoch = convert_epoch_object_events(epoch_object=epoch.copy())
        epochs_list.append(new_epoch)
        total_bytes += sys.getsizeof(new_epoch.get_data())
        print("total_bytes:", total_bytes)
#         print(new_epoch.events)
# master_epochs = mne.epochs.concatenate_epochs(epochs_list)

In [4]:
combined_master_list_epochs = []
for i in range(len(master_epochs_list)):
    combined_master_list_epochs.append(mne.epochs.concatenate_epochs(master_epochs_list[i]))
 

In [ ]:
sub_1_master_epochs = mne.epochs.concatenate_epochs(combined_master_list_epochs[:3])

In [5]:
sub_2_master_epochs = mne.epochs.concatenate_epochs([combined_master_list_epochs[0], combined_master_list_epochs[1]])

33012 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 bad epochs dropped


In [ ]:
new_events = convert_epoch_events_to_stim_combinations(master_epochs)
master_epochs.events = new_events

In [ ]:
# drop bad channels 
if 'Nasium' in master_epochs.ch_names:
    master_epochs.drop_channels(ch_names=['Nasium', 'LL4', 'L12', 'VEOG']);

In [ ]:
def get_audio_visual_labels(epoch_object):
    events = epoch_object.events[:,-1]
    labels = []
    for event in events:
        if event < 700:
            labels.append(0)
        else:
            labels.append(1)
    return np.array(labels)

In [ ]:
def get_frequency_band(band, epoch_object):
    """
    Description:
        returns an epoch object with a filtered frequency band
    
    Variables: 
        band : 
            'Theta':(4,7),
            'Alpha':(8,12),
            'Beta':(13,25),
            'Gamma':(30,45)
        epoch_object: mne.epoch
    -----
    returns: epoch object
    
    """
    iter_freqs = {
        'Theta':(4,7),
        'Alpha':(8,12),
        'Beta':(13,25),
        'Gamma':(30,45)
    }
    copy_epoch = epoch_object.copy()
    copy_epoch.filter(l_freq= iter_freqs[band][0],
                    h_freq = iter_freqs[band][1])
    
    return copy_epoch

In [ ]:
def classify_lgr(epoch_object, data_range, do_mean_stats = False, freq_band ='None'):
    clf = make_pipeline(Vectorizer(),
                        StandardScaler(),
                        LinearModel(LogisticRegression(solver='lbfgs',max_iter=1000)))
    start_master = time.time()
    new_events = convert_epoch_events_to_stim_combinations(epoch_object= epoch_object)
    
    epoch_object.events = new_events
    labels = get_audio_visual_labels(epoch_object)
    
    start_master = time.time()
    cv = StratifiedKFold(n_splits=5, shuffle = True, random_state = 42)
    
    if freq_band.lower() == 'gamma':
        freq_data = get_frequency_band('Gamma', epoch_object=epoch_object)
    elif freq_band.lower() == 'alpha':
        freq_data = get_frequency_band('Alpha', epoch_object=epoch_object)
    elif freq_band.lower() == 'beta':
        freq_data = get_frequency_band('Beta', epoch_object=epoch_object)
    elif freq_band.lower() == 'theta':
        freq_data = get_frequency_band('Theta', epoch_object=epoch_object)
    elif freq_band.lower() =='none':
        freq_data = epoch_object.get_data()
    else:
        return "{0} is an invalid frequency band! \n Please select \
                Theta, Alpha, Beta, or Gamma".format(freq_band)
        
    
    if do_mean_stats:
        data = get_mean_band(freq_data)
        data = data.swapaxes(1,2)
    else:
        data = freq_data
    
    data = data[:,:,data_range[0]:data_range[1]]
    print(data.shape)
    print(labels.shape)
    preds = np.empty(len(data))
    
    for train, test in cv.split(data, labels):
        start = time.time()
        clf.fit(data[train], labels[train])
        preds[test] = clf.predict(data[test])
        try:
            print(clf.score(data[test], labels[test]))
        except:
            print("could not get score")
        end = time.time()
        print("kfold elapsed time: {0}".format(end - start))
    end = time.time()
    print("classification elapsed time: {0}".format(end - start_master))

    start = time.time()
    target_names = ['Audio', 'Visual']
    report = classification_report(labels, preds, target_names=target_names)
    print(report)
    end = time.time()
    print("reporting elapsed time: {0}".format(end - start))
    
    return report

In [ ]:
audioVisualLabels = get_audio_visual_labels(master_epochs)

In [ ]:
print(len([x for x in audioVisualLabels if x == 0]))
len([x for x in audioVisualLabels if x == 1])

In [ ]:
len(audioLabels)

In [ ]:
try:
    classify_lgr(epoch_object=master_epochs.copy(),data_range=[0,300],  do_mean_stats=False, freq_band='None')
except:
    print("Oops, something went wrong!")

In [ ]:
try:
    classify_lgr(epoch_object=master_epochs.copy(),data_range=[0,300],  do_mean_stats=True, freq_band='None')
except:
    print("Oops, something went wrong!")

In [ ]:
try:
    classify_lgr(epoch_object=master_epochs.copy(),data_range=[0,300],  do_mean_stats=False, freq_band='Alpha')
except:
    print("Oops, something went wrong!")

In [ ]:
try:
    classify_lgr(epoch_object=master_epochs.copy(),data_range=[0,300],  do_mean_stats=False, freq_band='Beta')
except:
    print("Oops, something went wrong!")

In [ ]:
try:
    classify_lgr(epoch_object=master_epochs.copy(),data_range=[0,300],  do_mean_stats=False, freq_band='Gamma')
except:
    print("Oops, something went wrong!")

In [ ]:
try:
    classify_lgr(epoch_object=master_epochs.copy(),data_range=[0,300],  do_mean_stats=False, freq_band='Theta')
except:
    print("Oops, something went wrong!")

In [ ]:
classify_lgr(epoch_object=master_epochs.copy(),data_range=[0,300],  do_mean_stats=True, freq_band='None')

In [ ]:
epoch = mne.read_epochs(file, preload= True)

In [ ]:
alpha_report = classify_lgr(epoch_object=epoch.copy(), do_mean_stats=False, freq_band='Alpha')

In [ ]:
alpha_report_mean_stats = classify_lgr(epoch_object=epoch.copy(), do_mean_stats=True, freq_band='alpha')

In [ ]:
beta_report = classify_lgr(epoch_object=epoch.copy(), do_mean_stats=False, freq_band='beta')
beta_report_mean_stats = classify_lgr(epoch_object=epoch.copy(), do_mean_stats=True, freq_band='beta')

In [ ]:
theta_report = classify_lgr(epoch_object=epoch.copy(), do_mean_stats=False, freq_band='theta')
theta_report_mean_stats = classify_lgr(epoch_object=epoch.copy(), do_mean_stats=True, freq_band='theta')

In [ ]:
gamma_report = classify_lgr(epoch_object=epoch.copy(), do_mean_stats=False, freq_band='gamma')
gamma_report_mean_stats = classify_lgr(epoch_object=epoch.copy(), do_mean_stats=True, freq_band='gamma')

In [ ]:
report_mean_stats = classify_lgr(epoch.copy(), do_mean_stats=True)

## Example of plotting coefficients from MNE

In [ ]:
# def classifyScore(epoch_object):
if 'Nasium' in epoch.ch_names:
    epoch.drop_channels(ch_names=['Nasium', 'LL4', 'L12', 'VEOG']);
    copy_epoch = epoch.copy()

X = copy_epoch.pick_types(meg=False, eeg=True)
# X = get_mean_band(X)
# X = X.swapaxes(1,2)
new_events = convert_epoch_events_to_stim_combinations(epoch_object= copy_epoch)    
copy_epoch.events = new_events
y = get_audio_visual_labels(copy_epoch)
X_train, X_test, y_train, y_test = train_test_split(X.get_data(), y, test_size = 0.3, random_state = 42)
# print(X.shape, y.shape)
print(X_train.shape, y_train.shape)
# Define a unique pipeline to sequentially:
clf = make_pipeline(
    Vectorizer(),                       # 1) vectorize across time and channels
    StandardScaler(),                   # 2) normalize features across trials
    LinearModel(
        LogisticRegression(solver='lbfgs',max_iter = 1000)))  # 3) fits a logistic regression

clf.fit(X_train, y_train)

# Extract and plot patterns and filters
for name in ('patterns_', 'filters_'):
    # The `inverse_transform` parameter will call this method on any estimator
    # contained in the pipeline, in reverse order.
    coef = get_coef(clf, name, inverse_transform=True)
    evoked = EvokedArray(coef, copy_epoch.info, tmin=copy_epoch.tmin)
    evoked.plot_topomap(title='EEG %s' % name[:-1], time_unit='s')

In [ ]:
if 'Nasium' in master_epochs.ch_names:
    master_epochs.drop_channels(ch_names=['Nasium', 'LL4', 'L12', 'VEOG']);
copy_epoch = master_epochs.copy()

X = copy_epoch.pick_types(meg=False, eeg=True)
# X = get_mean_band(X)
# X = X.swapaxes(1,2)
new_events = convert_epoch_events_to_stim_combinations(epoch_object= copy_epoch)    
copy_epoch.events = new_events
y = get_audio_visual_labels(copy_epoch)
# half_1_data = X.get_data()[:,:,round(.09*512):round(.140*512)]
# half_2_data = X.get_data()[:,:,round(.280*512):round(.330*512)]
# full_data = np.concatenate((half_1_data, half_2_data),axis = 2)
X_train, X_test, y_train, y_test = train_test_split(X.get_data(), y, test_size = 0.3, random_state = 42)
# print(X.shape, y.shape)
print(X_train.shape, y_train.shape)
# Define a unique pipeline to sequentially:
clf = make_pipeline(
    Vectorizer(),                       # 1) vectorize across time and channels
    StandardScaler(),                   # 2) normalize features across trials
    LinearModel(
        LogisticRegression(solver='lbfgs',max_iter = 1000)))  # 3) fits a logistic regression

clf.fit(X_train, y_train)

# Extract and plot patterns and filters
for name in ('patterns_', 'filters_'):
    # The `inverse_transform` parameter will call this method on any estimator
    # contained in the pipeline, in reverse order.
    coef = get_coef(clf, name, inverse_transform=True)
    evoked = EvokedArray(coef, copy_epoch.info, tmin=copy_epoch.tmin)
    evoked.plot_topomap(title='EEG %s' % name[:-1], time_unit='s', times=np.arange(0,.55,.05))

In [ ]:
if 'Nasium' in master_epochs.ch_names:
    master_epochs.drop_channels(ch_names=['Nasium', 'LL4', 'L12', 'VEOG']);
copy_epoch = master_epochs.copy()

X = copy_epoch.pick_types(meg=False, eeg=True)
# X = get_mean_band(X)
# X = X.swapaxes(1,2)
new_events = convert_epoch_events_to_stim_combinations(epoch_object= copy_epoch)    
copy_epoch.events = new_events
y = get_audio_visual_labels(copy_epoch)
# half_1_data = X.get_data()[:,:,round(.09*512):round(.140*512)]
# half_2_data = X.get_data()[:,:,round(.280*512):round(.330*512)]
# full_data = np.concatenate((half_1_data, half_2_data),axis = 2)
X_train, X_test, y_train, y_test = train_test_split(X.get_data(), y, test_size = 0.3, random_state = 42)
# print(X.shape, y.shape)
print(X_train.shape, y_train.shape)
# Define a unique pipeline to sequentially:
clf = make_pipeline(
    Vectorizer(),                       # 1) vectorize across time and channels
    StandardScaler(),                   # 2) normalize features across trials
    LinearModel(
        LogisticRegression(solver='lbfgs',max_iter = 5000, warm_start=True)))  # 3) fits a logistic regression

clf.fit(X_train, y_train)

# Extract and plot patterns and filters
for name in ('patterns_', 'filters_'):
    # The `inverse_transform` parameter will call this method on any estimator
    # contained in the pipeline, in reverse order.
    coef = get_coef(clf, name, inverse_transform=True)
    evoked = EvokedArray(coef, copy_epoch.info, tmin=copy_epoch.tmin)
    evoked.plot_topomap(title='EEG %s' % name[:-1], time_unit='s', times=np.arange(0,.55,.05))

In [ ]:

for name in ('patterns_', 'filters_'):
    # The `inverse_transform` parameter will call this method on any estimator
    # contained in the pipeline, in reverse order.
    coef = get_coef(clf, name, inverse_transform=True)
    evoked = EvokedArray(coef, copy_epoch.info, tmin=copy_epoch.tmin)
    evoked.plot_topomap(title='EEG %s' % name[:-1], time_unit='s', times=np.arange(0,.2,.01))

In [ ]:

for name in ('patterns_', 'filters_'):
    # The `inverse_transform` parameter will call this method on any estimator
    # contained in the pipeline, in reverse order.
    coef = get_coef(clf, name, inverse_transform=True)
    evoked = EvokedArray(coef, copy_epoch.info, tmin=copy_epoch.tmin)
    evoked.plot_topomap(title='EEG %s' % name[:-1], time_unit='s', times=np.arange(0.2,.4,.01))

In [ ]:
clf.score(X_test,y_test)

In [ ]:
coef.mean()

In [ ]:
if 'Nasium' in epoch.ch_names:
    epoch.drop_channels(ch_names=['Nasium', 'LL4', 'L12', 'VEOG']);
copy_epoch = epoch.copy()

X = copy_epoch.pick_types(meg=False, eeg=True)
X = get_mean_band(X)
X = X.swapaxes(1,2)
new_events = convert_epoch_events_to_stim_combinations(epoch_object= copy_epoch)    
copy_epoch.events = new_events
y = get_audio_visual_labels(copy_epoch)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)
# print(X.shape, y.shape)
print(X_train.shape, y_train.shape)
# Define a unique pipeline to sequentially:
clf = make_pipeline(
    Vectorizer(),                       # 1) vectorize across time and channels
    StandardScaler(),                   # 2) normalize features across trials
    LinearModel(
        LogisticRegression(solver='lbfgs',max_iter = 1000)))  # 3) fits a logistic regression

clf.fit(X_train, y_train)

# Extract and plot patterns and filters
for name in ('patterns_', 'filters_'):
    # The `inverse_transform` parameter will call this method on any estimator
    # contained in the pipeline, in reverse order.
    coef = get_coef(clf, name, inverse_transform=True)
    evoked = EvokedArray(coef, copy_epoch.info, tmin=copy_epoch.tmin)
    evoked.plot_topomap(title='EEG %s' % name[:-1], time_unit='s')

In [ ]:
half_1_data = X.get_data()[:,:,round(.09*512):round(.140*512)]
half_2_data = X.get_data()[:,:,round(.280*512):round(.330*512)]
full_data = np.concatenate((half_1_data, half_2_data),axis = 2)
full_data.shape

In [ ]:
clf.score(X_test,y_test)

In [ ]:
for data in coef:
    for numb in data:
        numb = numb - coef.std()
        

In [ ]:
coef.min()

In [ ]:
coef.min()

In [ ]:
coef.max()

In [ ]:
coef.std()

In [ ]:
epoch_evoked = epoch.average()
epoch_evoked.data.shape

In [ ]:
epoch_test = mne.read_epochs(os.path.join(path, epoch_files[2]), preload= True)

In [ ]:
copy = epoch_test.copy()
test_events = convert_epoch_events_to_stim_combinations(epoch_object= copy)

In [ ]:
copy.events = test_events

In [ ]:
y1_test = get_audio_visual_labels(copy)
y1_test.shape

In [ ]:
copy.events.shape

In [ ]:
if 'Nasium' in epoch_test.ch_names:
    epoch_test.drop_channels(ch_names=['Nasium', 'LL4', 'L12', 'VEOG']);
    
copy_epoch = epoch.copy()
test_copy_epoch = epoch_test.copy()
X_1 = test_copy_epoch.pick_types(meg=False, eeg=True)
X_1 = get_mean_band(X_1)
X_1 = X_1.swapaxes(1,2)
test_new_events = convert_epoch_events_to_stim_combinations(epoch_object= test_copy_epoch)    
test_copy_epoch.events = test_new_events
y_1 = get_audio_visual_labels(epoch_object=test_copy_epoch)
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_1, y_1, test_size = 0.3, random_state = 42)

In [ ]:
clf.fit(X_train_1,y_train_1);

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
clf.score(X_test, y_test)

In [ ]:
for name in ('patterns_', 'filters_'):
    # The `inverse_transform` parameter will call this method on any estimator
    # contained in the pipeline, in reverse order.
    coef = get_coef(clf, name, inverse_transform=True)
    evoked = EvokedArray(coef, copy_epoch.info, tmin=copy_epoch.tmin)
    evoked.plot_topomap(title='EEG %s' % name[:-1], time_unit='s', times = np.arange(0.0,.2,.010))

In [ ]:
for name in ('patterns_', 'filters_'):
    # The `inverse_transform` parameter will call this method on any estimator
    # contained in the pipeline, in reverse order.
    coef = get_coef(clf, name, inverse_transform=True)
    evoked = EvokedArray(coef, copy_epoch.info, tmin=copy_epoch.tmin)
    evoked.plot_topomap(title='EEG %s' % name[:-1], time_unit='s', times = np.arange(0.2,.4,.010))

In [ ]:
evoked.plot(time_unit ='ms');

In [ ]:
if 'Nasium' in epoch.ch_names:
    epoch.drop_channels(ch_names=['Nasium', 'LL4', 'L12', 'VEOG']);
copy_epoch = epoch.copy()

X = copy_epoch.pick_types(meg=False, eeg=True)
X = get_mean_band(X)
X = X.swapaxes(1,2)
new_events = convert_epoch_events_to_stim_combinations(epoch_object= copy_epoch)    
copy_epoch.events = new_events
y = get_audio_visual_labels(copy_epoch)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)
# print(X.shape, y.shape)
print(X_train.shape, y_train.shape)
# Define a unique pipeline to sequentially:
clf = make_pipeline(
    Vectorizer(),                       # 1) vectorize across time and channels
    StandardScaler(),                   # 2) normalize features across trials
    LinearModel(
        LogisticRegression(solver='lbfgs',max_iter = 1000)))  # 3) fits a logistic regression

clf.fit(X_train, y_train)

# Extract and plot patterns and filters
for name in ('patterns_', 'filters_'):
    # The `inverse_transform` parameter will call this method on any estimator
    # contained in the pipeline, in reverse order.
    coef = get_coef(clf, name, inverse_transform=True)
    evoked = EvokedArray(coef, copy_epoch.info, tmin=copy_epoch.tmin)
    evoked.plot_topomap(title='EEG %s' % name[:-1], time_unit='s')
    evoked.plot_topo()

In [ ]:
clf.score(X_test,y_test)

In [ ]:
if 'Nasium' in epoch.ch_names:
    epoch.drop_channels(ch_names=['Nasium', 'LL4', 'L12', 'VEOG']);
copy_epoch = epoch.copy()


In [ ]:
if 'Nasium' in epoch.ch_names:
    epoch.drop_channels(ch_names=['Nasium', 'LL4', 'L12', 'VEOG']);
copy_epoch = epoch.copy()

copy_epoch = epoch.copy()
X = copy_epoch.pick_types(meg=False, eeg=True)
new_events = convert_epoch_events_to_stim_combinations(epoch_object= copy_epoch)

copy_epoch.events = new_events

y = get_audio_visual_labels(copy_epoch)
clf = make_pipeline(StandardScaler(),
                    LinearModel(LogisticRegression(solver='lbfgs', max_iter = 500)))
time_decod = SlidingEstimator(clf, n_jobs=1, scoring='roc_auc')

time_decod.fit(X.get_data(), y)

coef = get_coef(time_decod, 'patterns_', inverse_transform=True)
evoked = mne.EvokedArray(coef, copy_epoch.info, tmin=copy_epoch.times[0])
joint_kwargs = dict(ts_args=dict(time_unit='s'),
                    topomap_args=dict(time_unit='s'))
evoked.plot_joint(times=np.arange(0., .600, .100), title='patterns',
                  **joint_kwargs)

In [ ]:
evoked.plot_joint(times=np.arange(0., .600, .100), title='patterns',
                  **joint_kwargs);

In [ ]:
# We will train the classifier on all left visual vs auditory trials on MEG

clf = make_pipeline(StandardScaler(), LogisticRegression(solver='lbfgs', max_iter = 500))

time_decod = SlidingEstimator(clf, n_jobs=1, scoring='roc_auc')
scores = cross_val_multiscore(time_decod, X.get_data(), y, cv=5, n_jobs=1)

# Mean scores across cross-validation splits
scores = np.mean(scores, axis=0)

# Plot
fig, ax = plt.subplots()
ax.plot(copy_epoch.times, scores, label='score')
ax.axhline(.5, color='k', linestyle='--', label='chance')
ax.set_xlabel('Times')
ax.set_ylabel('AUC')  # Area Under the Curve
ax.legend()
ax.axvline(.0, color='k', linestyle='-')
ax.set_title('Sensor space decoding')

In [ ]:
# Plot
fig, ax = plt.subplots(figsize=(10,8))
ax.plot(copy_epoch.times, scores, label='score')
ax.axhline(.5, color='k', linestyle='--', label='chance')
ax.set_xlabel('Times (ms)', fontsize = 20)
ax.set_ylabel('AUC', fontsize = 20)  # Area Under the Curve
ax.legend()
ax.axvline(.0, color='k', linestyle='-')
ax.set_title('Sensor space decoding', fontsize = 20)

In [ ]:
matplotlib.__version__

In [ ]:
for name in ('patterns_', 'filters_'):
    # The `inverse_transform` parameter will call this method on any estimator
    # contained in the pipeline, in reverse order.
    coef = get_coef(clf, name, inverse_transform=True)
    evoked = EvokedArray(coef[:,0].reshape(-1,1), copy_epoch.info, tmin=copy_epoch.tmin)
    evoked.plot_topomap(title='EEG %s' % name[:-1], time_unit='ms', times= [0], res = 1000, size = 5, show_names=True, colorbar=False)

In [ ]:
variance_coef = coef[:,0]
variance_coef.shape

In [ ]:
np.arange(0,variance_coef.shape[0],1)

In [ ]:
feat_importance = np.std(X_train,0) * coef
print(feat_importance.shape)

In [ ]:
variance_feat_import = feat_importance[:,0]
plt.figure(figsize=(10,8))
plt.bar(np.arange(1,variance_feat_import.shape[0] +1,1),variance_feat_import, width = 1)
plt.title('Feature Importance of Logistic Regression Coefficients using Variance data', fontsize = 12)
plt.xlabel('Channels', fontsize = 15)
plt.ylabel('Uv', fontsize = 15)

In [ ]:
Skew_feat_import = feat_importance[:,1]
plt.figure(figsize=(10,8))
plt.bar(np.arange(1,variance_feat_import.shape[0] +1,1),Skew_feat_import, width = 1)
plt.title('Feature Importance of Logistic Regression Coefficients using Skewness data', fontsize = 12)
plt.xlabel('Channels', fontsize = 15)
plt.ylabel('Uv', fontsize = 15)

In [ ]:
Kurt_feat_import = feat_importance[:,2]
plt.figure(figsize=(10,8))
plt.bar(np.arange(1,variance_feat_import.shape[0] +1,1),Kurt_feat_import, width = 1)
plt.title('Feature Importance of Logistic Regression Coefficients using Kurtosis data', fontsize = 12)
plt.xlabel('Channels', fontsize = 15)
plt.ylabel('Uv', fontsize = 15)

In [ ]:
for name in ('patterns_', 'filters_'):
    # The `inverse_transform` parameter will call this method on any estimator
    # contained in the pipeline, in reverse order.
    coef = get_coef(clf, name, inverse_transform=True)
    evoked = EvokedArray(coef[:,1].reshape(-1,1), copy_epoch.info, tmin=copy_epoch.tmin)
    evoked.plot_topomap(title='EEG %s' % name[:-1], time_unit='ms', times= [0], size = 5,res = 1000, show_names=True, colorbar=False)

In [ ]:
for name in ('patterns_', 'filters_'):
    # The `inverse_transform` parameter will call this method on any estimator
    # contained in the pipeline, in reverse order.
    coef = get_coef(clf, name, inverse_transform=True)
    evoked = EvokedArray(coef[:,2].reshape(-1,1), copy_epoch.info, tmin=copy_epoch.tmin)
    evoked.plot_topomap(title='EEG %s' % name[:-1], time_unit='ms', times= [0],  show_names=True, size = 5,res = 1000, colorbar=False)
    evoked.plot_topo()

In [ ]:
coef_0 = coef[:,0].reshape(-1,1)
coef_1 = coef[:,1].reshape(-1,1)
coef_2 = coef[:,2].reshape(-1,1)

In [ ]:
plt.scatter(coef_1,coef_2)
# plt.xticks(np.arange)

In [ ]:
coef_1[y == 0,:]

In [ ]:
scale_pipeline = make_pipeline(Vectorizer(), StandardScaler())
X_scaled = scale_pipeline.fit_transform(X)

In [ ]:
vect = Vectorizer()

In [ ]:
X_scaled_vect = vect.fit_transform(X_scaled)

In [ ]:
X_scaled_vect.shape

In [ ]:
plt.scatter(X[:,:,1][y == 1], X[:,:,2][y == 1]);
plt.scatter(X[:,:,1][y == 0], X[:,:,2][y == 0]);


In [ ]:
coef[:,1].reshape(-1,1).shape

### Based on the above diagrams, classify time between 100 - 350 ms

In [ ]:
copy_epoch = epoch.copy()
report_100ms_350ms = classify_lgr(copy_epoch, data_range=[51,180] do_mean_stats=False)

In [ ]:
copy_epoch = epoch.copy()
report_all = classify_lgr(copy_epoch, data_range=[0,300], do_mean_stats=False)

In [ ]:
print(gamma_report)

In [ ]:
updated_events = convert_epoch_events_to_stim_combinations(epoch_object=epoch.copy())

In [ ]:
updated_events.shape

In [ ]:
file_1 = os.path.join(path, epoch_files[19])
file_2 = os.path.join(path, epoch_files[10])
file_3 = os.path.join(path, epoch_files[2])

In [ ]:
epoch_1 = mne.read_epochs(file_1, preload= True)
epoch_2 = mne.read_epochs(file_2, preload= True)
epoch_3 = mne.read_epochs(file_3, preload= True)

In [ ]:
# drop bad channels 
if 'Nasium' in epoch_1.ch_names:
    epoch_1.drop_channels(ch_names=['Nasium', 'LL4', 'L12', 'VEOG']);
    
# drop bad channels 
if 'Nasium' in epoch_2.ch_names:
    epoch_2.drop_channels(ch_names=['Nasium', 'LL4', 'L12', 'VEOG']);
    
# drop bad channels 
if 'Nasium' in epoch_3.ch_names:
    epoch_3.drop_channels(ch_names=['Nasium', 'LL4', 'L12', 'VEOG']);

In [ ]:
report_1 = classify_lgr(epoch_object=epoch_1.copy(), data_range= [0,300],do_mean_stats=True)
report_2 = classify_lgr(epoch_object=epoch_2.copy(), data_range= [0,300],do_mean_stats=True)
report_3 = classify_lgr(epoch_object=epoch_3.copy(), data_range= [0,300],do_mean_stats=True)

In [ ]:
report_2 = classify_lgr(epoch_object=epoch_2.copy(), data_range= [0,300],do_mean_stats=True)

In [ ]:
report_3 = classify_lgr(epoch_object=epoch_3.copy(), data_range= [0,300],do_mean_stats=True)

In [ ]:
subjects = ['19','10','12']
scores_1 = np.array([.73,.67])
scores_2 = [.74,.77]
scores_3 = [.69,.66]
plt.figure(figsize=(8,6))
ax = plt.subplot()

ax.bar('subject 19', scores_1, width = 0.5,align='center')
ax.bar('subject 10', scores_2, width = 0.5,align='center')
ax.bar('subject 2', scores_3, width = 0.5, align='center')
ax.set_title('Classifcation scores of three subjects', fontsize = 20)
ax.set_xlabel('Subjects', fontsize = 20)
ax.set_ylabel('Score',fontsize = 20)
ax.grid(axis = 'y')
plt.show();

In [ ]:
plt.bar()

In [ ]:
file_2 = os.path.join(path, epoch_files[10])

In [ ]:
epoch_2 = mne.read_epochs(file_2, preload= True)

In [ ]:
epoch_2.get_data().shape

In [ ]:
epoch_2.events.shape

In [ ]:
epoch_1.plot_psd_topomap();

In [ ]:
epoch_2.plot_psd_topomap();

In [ ]:
epoch_3.plot_psd_topomap();

In [ ]:
sys.getsizeof(epoch.get_data())

In [ ]:
master_epochs_list

In [ ]:
len(epochs_list)